In [1]:
import os
import sys
import random
from pathlib import Path
import osgeo  #Needed for use on Windows only
import matplotlib.pyplot as plt
from rasterio import plot
from rasterio.plot import show
import geopandas as gpd
from datetime import datetime
import sqlalchemy as sa
import pandas as pd
%matplotlib inline

In [2]:
sys.path.append(r"../../src/collectcube")
from samplegen import *
from db import *

In [3]:
out_dir = 'C:/GISprojects/ParaguayTraining'
local_db_path = os.path.join(out_dir, 'py_training.db')
## reference .tif with target crs and resolution
ref_ras = Path("../../data/CEL_py/samp_ras.tif")

res = 10

## Create new pixel table from scratch

In [ ]:
 # see create_db notebook

## Format existing points

### Get max pid in current table (to assign new ids to points to add)

In [102]:
maxpid = get_max_id_in_db(local_db_path)
print(f'current max pid is {maxpid}')

current max pid is 31471


Need to make sure that the new pixel data has all of the same fields as the existing table. Need to pre-fill PID0 (starting with current max from above + 1), PID1 (0 usually for new points), PID (concat(PID,"_"PID1), cent_X & cent_Y (in project crs - EPSG 8858 here), and sampgroup. Save as CSV file to load below.

In [53]:
old_pts = pd.read_csv(os.path.join(out_dir,'lastpts.csv'))
old_pts_df = gpd.GeoDataFrame(old_pts,geometry = gpd.points_from_xy(old_pts.cent_X, old_pts.cent_Y, crs="EPSG:8858"))
print(old_pts_df[old_pts_df['PID0'] == 31449])

        PID  Center   cent_X   cent_Y  cent_lat  cent_long  ransamp  checked  \
12  31449_0       1  3070014 -3048529       NaN        NaN        0        0   

     PID0  PID1 sampGroup      rand    rand2                          geometry  
12  31449     0  GE_extra  0.301958  0.64626  POINT (3070014.000 -3048529.000)  


In [54]:
old_pts_df.to_crs('4326')
old_pts_df['cent_long'] = old_pts_df['geometry'].x
old_pts_df['cent_lat'] = old_pts_df['geometry'].y
old_pts_df.to_crs("EPSG:8858")
print(old_pts_df[old_pts_df['PID0'] == 31449])

        PID  Center   cent_X   cent_Y   cent_lat  cent_long  ransamp  checked  \
12  31449_0       1  3070014 -3048529 -3048529.0  3070014.0        0        0   

     PID0  PID1 sampGroup      rand    rand2                          geometry  
12  31449     0  GE_extra  0.301958  0.64626  POINT (3070014.000 -3048529.000)  


In [55]:
old_pts_df.head()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampGroup,rand,rand2,geometry
0,31437_0,1,3101210,-3284257,-3284257.0,3101210.0,0,0,31437,0,GE_extra,0.658017,0.304797,POINT (3101210.000 -3284257.000)
1,31438_0,1,3101290,-3284260,-3284260.0,3101290.0,0,0,31438,0,GE_extra,0.323210,0.861102,POINT (3101290.000 -3284260.000)
2,31439_0,1,3101063,-3284535,-3284535.0,3101063.0,0,0,31439,0,GE_extra,0.964859,0.063600,POINT (3101063.000 -3284535.000)
3,31440_0,1,3101161,-3284440,-3284440.0,3101161.0,0,0,31440,0,GE_extra,0.090288,0.277210,POINT (3101161.000 -3284440.000)
4,31441_0,1,3100439,-3284192,-3284192.0,3100439.0,0,0,31441,0,GE_extra,0.912895,0.427364,POINT (3100439.000 -3284192.000)


In [56]:
ptfile = os.path.join(out_dir,'lastPtsLAST_cent.shp')
centered_pts = move_points_to_pixel_centroids(old_pts_df, ref_ras, write_pts=True, ptsout=ptfile)

ref_ras has crs:EPSG:8858


In [57]:
print(centered_pts.loc[31449])

geometry    POINT (3070020.000 -3048540.000)
Name: 31449, dtype: geometry


In [ ]:
all_pts = get_full_point_file(centered_pts, pt_file_out=None, res=res, lastpt=-1, write_pts=False)
all_pts.head()

In [60]:
boxfile = os.path.join(out_dir,'training_boxes_lastPtsLAST.shp')
make_pixel_boxes_from_pts(all_pts, boxfile, res, write_boxes=True)

,geometry
0,"POLYGON ((3101225.000 -3284265.000, 3101225.00..."
1,"POLYGON ((3101305.000 -3284265.000, 3101305.00..."
2,"POLYGON ((3101075.000 -3284545.000, 3101075.00..."
3,"POLYGON ((3101175.000 -3284445.000, 3101175.00..."
4,"POLYGON ((3100455.000 -3284195.000, 3100455.00..."
...,...
310,"POLYGON ((3145075.000 -3123505.000, 3145075.00..."
311,"POLYGON ((3145095.000 -3123505.000, 3145095.00..."
312,"POLYGON ((3145075.000 -3123515.000, 3145075.00..."
313,"POLYGON ((3145085.000 -3123515.000, 3145085.00..."


In [61]:
pts_info = make_pixel_table(all_pts,samp_group=None)
pts_info.drop(columns=['sampgroup'], inplace=True)
#old_pts.rename({'sampGroup': 'sampgroup'}, axis='columns', inplace=True)  #Fixing mistake
pts_info2 = pd.merge(pts_info,old_pts[['PID0','sampgroup','rand','rand2']],on='PID0', how='inner')
pts_info2.head()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2
0,0031437_0,1,3101220.0,-3284260.0,-26.016351,-55.981892,1,0,31437,0,GE_extra,0.658017,0.304797
1,0031437_1,0,3101210.0,-3284250.0,-26.016269,-55.982013,1,0,31437,1,GE_extra,0.658017,0.304797
2,0031437_2,0,3101220.0,-3284250.0,-26.016269,-55.981903,1,0,31437,2,GE_extra,0.658017,0.304797
3,0031437_3,0,3101230.0,-3284250.0,-26.016269,-55.981793,1,0,31437,3,GE_extra,0.658017,0.304797
4,0031437_4,0,3101210.0,-3284260.0,-26.016351,-55.982002,1,0,31437,4,GE_extra,0.658017,0.304797


In [100]:
#print(pts_info2[pts_info2['PID0'] == 31449])

### To append new pixels to existing table (new pixels formatted above) 

In [ ]:
make_pixel_table_in_db(pts_info2, local_db_path, treat_existing='append')
table_check = check_table(local_db_path,'pixels')

### To update existing records in table

###### Adding new fields
For example here we want to add a field called gridCell containing the value of the 'gridCell' (from a polygon file) each point falls in and a field called 'biome' containing the biome (from a raster file) each point falls in

In [ ]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE pixels ADD COLUMN biome TEXT")
con.commit()
c.execute("ALTER TABLE pixels ADD COLUMN gridCell INTEGER")
con.commit()
c.close()

In [71]:
## Load pixel table
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM pixels", conn)
db_df.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2
277411,0031471_4,0,3145070.0,-3123500.0,-24.698683,-55.671875,1,0,31471,4,GE_extra,0.110972,0.142119
277412,0031471_5,0,3145090.0,-3123500.0,-24.698683,-55.671657,1,0,31471,5,GE_extra,0.110972,0.142119
277413,0031471_6,0,3145070.0,-3123510.0,-24.698765,-55.671865,1,0,31471,6,GE_extra,0.110972,0.142119
277414,0031471_7,0,3145080.0,-3123510.0,-24.698765,-55.671756,1,0,31471,7,GE_extra,0.110972,0.142119
277415,0031471_8,0,3145090.0,-3123510.0,-24.698765,-55.671647,1,0,31471,8,GE_extra,0.110972,0.142119


In [72]:
## To fill the biome field

def get_val_at_XY(img,df,colname):
    coords = [(x,y) for x, y in zip(df.cent_X, df.cent_Y)]
    with rio.open(img) as src:
        pts = [x[0] for x in src.sample(coords)]
    df[colname] = pts

    return df

forstrat = 'C:/Users/klobw/Desktop/NasaProject/Paraguay/GIS_datasets/forest_strata3.tif'
vals = get_val_at_XY(forstrat,db_df,'biome')
forstrat_key = {1:'Chaco', 2:'BS',3:'Cer', 4:'BH'}
vals['biome'] = vals['biome'].replace(forstrat_key)
vals.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,biome
277411,0031471_4,0,3145070.0,-3123500.0,-24.698683,-55.671875,1,0,31471,4,GE_extra,0.110972,0.142119,BH
277412,0031471_5,0,3145090.0,-3123500.0,-24.698683,-55.671657,1,0,31471,5,GE_extra,0.110972,0.142119,BH
277413,0031471_6,0,3145070.0,-3123510.0,-24.698765,-55.671865,1,0,31471,6,GE_extra,0.110972,0.142119,BH
277414,0031471_7,0,3145080.0,-3123510.0,-24.698765,-55.671756,1,0,31471,7,GE_extra,0.110972,0.142119,BH
277415,0031471_8,0,3145090.0,-3123510.0,-24.698765,-55.671647,1,0,31471,8,GE_extra,0.110972,0.142119,BH


In [73]:
## To fill the gridCell field

def get_val_at_XY_poly(poly_file,df,colname):
    polys = gpd.read_file(poly_file)
    points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=df.cent_X, y=df.cent_Y), crs=polys.crs)
    sj = gpd.sjoin(left_df=points, right_df=polys, how="left", rsuffix="ply_")
    df[colname]=sj['UNQ'].astype('Int64')
    return df

grids = 'C:/Users/klobw/Desktop/NasaProject/LUCinLA_grid_8858.gpkg'
vals2 = get_val_at_XY_poly(grids,vals,'gridCell')
vals2.tail()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,biome,gridCell
277411,0031471_4,0,3145070.0,-3123500.0,-24.698683,-55.671875,1,0,31471,4,GE_extra,0.110972,0.142119,BH,3972
277412,0031471_5,0,3145090.0,-3123500.0,-24.698683,-55.671657,1,0,31471,5,GE_extra,0.110972,0.142119,BH,3972
277413,0031471_6,0,3145070.0,-3123510.0,-24.698765,-55.671865,1,0,31471,6,GE_extra,0.110972,0.142119,BH,3972
277414,0031471_7,0,3145080.0,-3123510.0,-24.698765,-55.671756,1,0,31471,7,GE_extra,0.110972,0.142119,BH,3972
277415,0031471_8,0,3145090.0,-3123510.0,-24.698765,-55.671647,1,0,31471,8,GE_extra,0.110972,0.142119,BH,3972


In [69]:
#vals2.to_csv(os.path.join('C:/GISprojects/ParaguayTraining','allPix.csv'), index=False)

###### Updating data in existing fields from Pandas db

In [ ]:
## Note this is slow if completely filling a new column 
##   but better than deleting all records because of auto-incremented record number

def update_data(df,update_col,con)
    c = con.cursor()
    s_update = ""
    for i in range(len(df)):
        s_update += "update pixels set biome = '%s' where PID = '%s';"%(df[update_col][i], df['PID'][i])
    c.executescript(s_update)
    con.commit()
    c.close()
    
con = sqlite3.connect(local_db_path)
## NOTE need to repace column name in the method above. TODO fix this
#update_data(vals2,'biome',con)

###### Updating data in existing fields with arguments

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET cent_X = '2772120.0' WHERE PID0 = '30387'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2013-07-23' WHERE imgDate ='7/23/2013'")
con.commit()
c.close()
'''

###### Deleting records -- WATCH OUT - there is not recovery after commiting

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM pixels WHERE sampgroup = 'Ground_AtaValid'")
con.commit()
c.close()
'''

###### Updating lat long coords after adjusting cent_X or cent_Y
(When viewing a point, it is sometimes clear that the point does not quite fall within the field / land cover it is labeled for (this is especially common in ground data collected from the road). The point can easily be moved by updating cent_X or cent_Y with the methods above if these are in meters/feet (taking care to move in increments matching the pixel resolution so that the new point will be at a pixel center). After such adjustments, the lat/long fields need adjusting as well (this is better done in larger batches at the end of any editing sessions)   

In [101]:
## Load pixel table
#conn = sqlite3.connect(local_db_path, isolation_level=None,
#    detect_types=sqlite3.PARSE_COLNAMES)
#df = pd.read_sql_query("SELECT * FROM pixels", conn)

gdf = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.cent_X, df.cent_Y, crs="EPSG:8858"))
gdfll = gdf.to_crs('4326')
#print(gdfll[gdfll['PID0'] == 30387])
gdfll['cent_long'] = gdfll['geometry'].x
gdfll['cent_lat'] = gdfll['geometry'].y
gdfutm = gdfll.to_crs("EPSG:8858")
gdfutm.drop(columns=['geometry'],inplace=True)
#print(gdfutm[gdfutm['PID0'] == 30387])
#gdfutm.to_csv(os.path.join('D:/NasaProject/temp_backup','allPix_adjusted.csv'), index=False)

# Now use method in "Updating data in existing fields from Pandas db" above to update cent_long and cent_lat fields from gdfutm

## Make other db tables

In [10]:
lc_lut = Path("../../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')
make_simp_LC_table(lc_lut, local_db_path, [0,30,35,40,80,98], treat_existing='fail')
make_db(local_db_path)

added LC5 and PixVar tables to db


## checks:

#### inspect a single record

In [ ]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification WHERE recID = '3573'")
con.commit()
print(c.fetchone()) 
c.close()

#### Make sure table looks right and is correctly formatted

In [ ]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as con:
    df = pd.read_sql_table('pixels', con)
df.head()

In [11]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------'), (10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


In [99]:
#make_pixel_table_in_db(gdfutm, local_db_path, treat_existing='append')
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277411  0031471_4       0  3145070.0 -3123500.0 -24.698683 -55.671875   
277412  0031471_5       0  3145090.0 -3123500.0 -24.698683 -55.671657   
277413  0031471_6       0  3145070.0 -3123510.0 -24.698765 -55.671865   
277414  0031471_7       0  3145080.0 -3123510.0 -24.698765 -55.671756   
277415  0031471_8       0  3145090.0 -3123510.0 -24.698765 -55.671647   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277411        0        0  31471     4  GE_extra  0.110972  0.142119    BH   
277412        0        0  31471     5  GE_extra  0.110972  0.142119    BH   
277413        0        0  31471     6  GE_extra  0.110972  0.142119    BH   
277414        0        0  31471     7  GE_extra  0.110972  0.142119    BH   
277415        0        0  31471     8  GE_extra  0.110972  0.142119    BH   

        gridCell  
277411      3972  
277412      3972  
277413      3972  
277414      3972  
277

If the above fails, can convert to pandas df and export to find errors (then update with the methods above to fix)

In [65]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM pixels", conn)
db_df.tail()
#db_df.to_csv(os.path.join('C:/GISprojects/ParaguayTraining','allPix.csv'), index=False)

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2
277411,0031471_4,0,3145070.0,-3123500.0,-24.698683,-55.671875,1,0,31471,4,GE_extra,0.110972,0.142119
277412,0031471_5,0,3145090.0,-3123500.0,-24.698683,-55.671657,1,0,31471,5,GE_extra,0.110972,0.142119
277413,0031471_6,0,3145070.0,-3123510.0,-24.698765,-55.671865,1,0,31471,6,GE_extra,0.110972,0.142119
277414,0031471_7,0,3145080.0,-3123510.0,-24.698765,-55.671756,1,0,31471,7,GE_extra,0.110972,0.142119
277415,0031471_8,0,3145090.0,-3123510.0,-24.698765,-55.671647,1,0,31471,8,GE_extra,0.110972,0.142119


In [84]:
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification WHERE recID = '3573'")
con.commit()
print(c.fetchone()) 
c.close()

(3573, '3648_0', 3648, 0, '2022-01-01', 50, 52, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ' ', 1, None, 'GE')


## cleaning

In [98]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2013-07-23' WHERE imgDate ='7/23/2013'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET cent_X = '2772120.0' WHERE PID0 = '30387'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM pixels WHERE sampgroup = 'Ground_AtaValid'")
con.commit()
c.close()
'''

'\ncon = sqlite3.connect(local_db_path)\nc = con.cursor()\nc.execute("DELETE FROM pixels WHERE sampgroup = \'Ground_AtaValid\'")\ncon.commit()\nc.close()\n'